# Is there any relationship between district investments, vehicles sales and stamps revenue within the same district between FY 2021 and 2022?

In [18]:
import yaml
import mysql.connector
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import seaborn as sns
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import squarify
import geopandas as gpd
import matplotlib.patches as mpatches
import matplotlib.colors as mcolors
import folium

import scipy.stats as stats
from scipy.stats import ks_2samp
from scipy.stats import kstest, norm
from scipy.stats import levene
import statsmodels.api as sm
from statsmodels.tsa.stattools import grangercausalitytests
from sklearn.preprocessing import MinMaxScaler

import warnings
warnings.filterwarnings('ignore')
with warnings.catch_warnings():
    warnings.simplefilter("ignore")

In [3]:
# Load the YAML file
with open("D:\Telangana_Growth_Analysis\db_config.yaml", 'r') as file:
    db_config = yaml.safe_load(file)

# Connect to the database
connection = mysql.connector.connect(**db_config)
cursor = connection.cursor()

In [4]:
# SQL query to join 'fact_stamps' and 'dim_districts' on 'dist_code'

query = """
    select * from documents
    inner join districts on documents.dist_code = districts.dist_code
    """
# Use pandas to run the query and store the result in a DataFrame
stamps = pd.read_sql(query, connection)

# Sort the DataFrame by the 'id' column in ascending order
stamps.sort_values(by='id', ascending=True, inplace=True)
stamps.reset_index(drop=True, inplace=True)


# Load the 'dim_date' table into a DataFrame
dim_date = pd.read_sql("SELECT * FROM dim_date", connection)

# Merge the 'stamps' DataFrame with the 'dim_date' DataFrame on the 'month' column
stamps = pd.merge(stamps, dim_date, on='month', how='inner')
# Display the first few rows of the merged DataFrame
stamps.head()

,id,dist_code,month,documents_registered_cnt,documents_registered_rev,estamps_challans_cnt,estamps_challans_rev,dist_code,district,mmm,quarter,fiscal_year
0,1,14_1,2019-04-01,4533,59236363.0,0,0.0,14_1,Mahabubnagar\r,Apr,Q1,2019
1,2,17_3,2019-04-01,4151,41508762.0,0,0.0,17_3,Siddipet\r,Apr,Q1,2019
2,3,20_3,2019-04-01,2116,23674170.0,0,0.0,20_3,Rajanna Sircilla\r,Apr,Q1,2019
3,4,21_5,2019-04-01,1089,15915285.0,0,0.0,21_5,Mahabubabad\r,Apr,Q1,2019
4,5,23_1,2019-04-01,6133,82593256.0,0,0.0,23_1,Nalgonda\r,Apr,Q1,2019


In [5]:
# SQL query to join 'fact_stamps' and 'dim_districts' on 'dist_code'

query = """
    select * from vehicles
    inner join districts on vehicles.dist_code = districts.dist_code
    """
# Use pandas to run the query and store the result in a DataFrame
vehicles = pd.read_sql(query, connection)

# Sort the DataFrame by the 'id' column in ascending order
vehicles.sort_values(by='id', ascending=True, inplace=True)
vehicles.reset_index(drop=True, inplace=True)


# Load the 'dim_date' table into a DataFrame
dim_date = pd.read_sql("SELECT * FROM dim_date", connection)

# Merge the 'vehicles' DataFrame with the 'dim_date' DataFrame on the 'month' column
vehicles = pd.merge(vehicles, dim_date, on='month', how='inner')
# Display the first few rows of the merged DataFrame

# Set Pandas display options to show all columns
pd.set_option('display.max_columns', None)
vehicles.head()

,id,dist_code,month,fuel_type_petrol,fuel_type_diesel,fuel_type_electric,fuel_type_others,vehicleClass_MotorCycle,vehicleClass_MotorCar,vehicleClass_AutoRickshaw,vehicleClass_Agriculture,vehicleClass_others,seatCapacity_1_to_3,seatCapacity_4_to_6,seatCapacity_above_6,brand_new_vehicles,pre_owned_vehicles,category_non_transport,category_transport,dist_code,district,mmm,quarter,fiscal_year
0,1,15_1,2019-04-01,17910,3011,76,22,15308,4429,0,4,1278,16110,4182,717,19542,1477,19856,1163,15_1,Rangareddy\r,Apr,Q1,2019
1,2,18_2,2019-04-01,3066,306,6,0,2995,142,49,64,128,3156,189,33,3322,56,3203,175,18_2,Kamareddy\r,Apr,Q1,2019
2,3,20_3,2019-04-01,1577,215,0,0,1546,79,29,21,117,1683,104,5,1751,41,1648,144,20_3,Rajanna Sircilla\r,Apr,Q1,2019
3,4,21_3,2019-04-01,1961,281,2,0,1939,72,72,48,113,2082,146,16,2209,35,2075,169,21_3,Jangoan\r,Apr,Q1,2019
4,5,21_7,2019-04-01,1552,309,0,0,1512,76,69,109,95,1696,145,20,1820,41,1701,160,21_7,Jayashankar Bhupalpally\r,Apr,Q1,2019


In [6]:
# SQL query to join 'fact_stamps' and 'dim_districts' on 'dist_code'

query = """
    select * from investments
    inner join districts on investments.dist_code = districts.dist_code
    """
# Use pandas to run the query and store the result in a DataFrame
investments = pd.read_sql(query, connection)

# Sort the DataFrame by the 'id' column in ascending order
investments.sort_values(by='id', ascending=True, inplace=True)
investments.reset_index(drop=True, inplace=True)


# Load the 'dim_date' table into a DataFrame
dim_date = pd.read_sql("SELECT * FROM dim_date", connection)

# Merge the 'vehicles' DataFrame with the 'dim_date' DataFrame on the 'month' column
investments = pd.merge(investments, dim_date, on='month', how='inner')
# Display the first few rows of the merged DataFrame

# Set Pandas display options to show all columns
pd.set_option('display.max_columns', None)
investments.head()

,id,dist_code,month,sector,investment_in_cr,number_of_employees,dist_code,district,mmm,quarter,fiscal_year
0,1,14_1,2019-04-01,Engineering,2.32,15,14_1,Mahabubnagar\r,Apr,Q1,2019
1,2,19_1,2019-04-01,Engineering,0.63,13,19_1,Adilabad\r,Apr,Q1,2019
2,3,20_3,2019-04-01,Wood and Leather,0.20,8,20_3,Rajanna Sircilla\r,Apr,Q1,2019
3,4,20_3,2019-04-01,Textiles,0.27,27,20_3,Rajanna Sircilla\r,Apr,Q1,2019
4,5,21_5,2019-04-01,Electrical and Electronic Products,0.12,5,21_5,Mahabubabad\r,Apr,Q1,2019


In [7]:
#The \r is a carriage return character. 
# it can remove it from the district column using the str.replace method provided by pandas.

stamps['district'] = stamps['district'].str.replace('\r', '')

# drop dis_code
stamps.drop(columns=['dist_code','dist_code'], inplace=True)

vehicles['district'] = vehicles['district'].str.replace('\r', '')

# drop dis_code
vehicles.drop(columns=['dist_code','dist_code'], inplace=True)

investments['district'] = investments['district'].str.replace('\r', '')

# drop dis_code
investments.drop(columns=['dist_code','dist_code'], inplace=True)

In [8]:
# change date to datetime type

stamps['month'] = pd.to_datetime(stamps['month'])

vehicles['month'] = pd.to_datetime(vehicles['month'])

investments['month'] = pd.to_datetime(investments['month'])

In [9]:
stamps.head()

,id,month,documents_registered_cnt,documents_registered_rev,estamps_challans_cnt,estamps_challans_rev,district,mmm,quarter,fiscal_year
0,1,2019-04-01,4533,59236363.0,0,0.0,Mahabubnagar,Apr,Q1,2019
1,2,2019-04-01,4151,41508762.0,0,0.0,Siddipet,Apr,Q1,2019
2,3,2019-04-01,2116,23674170.0,0,0.0,Rajanna Sircilla,Apr,Q1,2019
3,4,2019-04-01,1089,15915285.0,0,0.0,Mahabubabad,Apr,Q1,2019
4,5,2019-04-01,6133,82593256.0,0,0.0,Nalgonda,Apr,Q1,2019


In [10]:
# Update the fiscal years to integers for filtering
stamps_df = stamps[stamps['fiscal_year'].isin([2021, 2022])]
investments_df = investments[investments['fiscal_year'].isin([2021, 2022])]
vehicles_df = vehicles[vehicles['fiscal_year'].isin([2021, 2022])]

# Merge the dataframes on 'month', 'district', and 'fiscal_year'
merged_df = pd.merge(stamps_df, investments_df, on=['month', 'district', 'fiscal_year'], how='inner')
merged_df = pd.merge(merged_df, vehicles_df, on=['month', 'district', 'fiscal_year'], how='inner')

# Now merged_df contains the merged data
merged_df.head()

,id_x,month,documents_registered_cnt,documents_registered_rev,estamps_challans_cnt,estamps_challans_rev,district,mmm_x,quarter_x,fiscal_year,id_y,sector,investment_in_cr,number_of_employees,mmm_y,quarter_y,id,fuel_type_petrol,fuel_type_diesel,fuel_type_electric,fuel_type_others,vehicleClass_MotorCycle,vehicleClass_MotorCar,vehicleClass_AutoRickshaw,vehicleClass_Agriculture,vehicleClass_others,seatCapacity_1_to_3,seatCapacity_4_to_6,seatCapacity_above_6,brand_new_vehicles,pre_owned_vehicles,category_non_transport,category_transport,mmm,quarter
0,769,2021-04-01,6310,92841579.0,6473,95086207.0,Yadadri Bhuvanagiri,Apr,Q1,2021,2860,"Fertlizers Organic and Inorganic,Pesticides,In...",2.50,14,Apr,Q1,741,1711,398,8,47,1408,330,50,215,161,1768,375,21,2084,80,1967,197,Apr,Q1
1,769,2021-04-01,6310,92841579.0,6473,95086207.0,Yadadri Bhuvanagiri,Apr,Q1,2021,2861,Food Processing,0.65,40,Apr,Q1,741,1711,398,8,47,1408,330,50,215,161,1768,375,21,2084,80,1967,197,Apr,Q1
2,769,2021-04-01,6310,92841579.0,6473,95086207.0,Yadadri Bhuvanagiri,Apr,Q1,2021,2867,Granite and Stone Crushing,6.25,29,Apr,Q1,741,1711,398,8,47,1408,330,50,215,161,1768,375,21,2084,80,1967,197,Apr,Q1
3,769,2021-04-01,6310,92841579.0,6473,95086207.0,Yadadri Bhuvanagiri,Apr,Q1,2021,2868,Paper and Printing,0.96,6,Apr,Q1,741,1711,398,8,47,1408,330,50,215,161,1768,375,21,2084,80,1967,197,Apr,Q1
4,769,2021-04-01,6310,92841579.0,6473,95086207.0,Yadadri Bhuvanagiri,Apr,Q1,2021,2870,Solar and Other Renewable Energy,1.12,40,Apr,Q1,741,1711,398,8,47,1408,330,50,215,161,1768,375,21,2084,80,1967,197,Apr,Q1


In [11]:
merged_df.columns

Index(['id_x', 'month', 'documents_registered_cnt', 'documents_registered_rev',
       'estamps_challans_cnt', 'estamps_challans_rev', 'district', 'mmm_x',
       'quarter_x', 'fiscal_year', 'id_y', 'sector', 'investment_in_cr',
       'number_of_employees', 'mmm_y', 'quarter_y', 'id', 'fuel_type_petrol',
       'fuel_type_diesel', 'fuel_type_electric', 'fuel_type_others',
       'vehicleClass_MotorCycle', 'vehicleClass_MotorCar',
       'vehicleClass_AutoRickshaw', 'vehicleClass_Agriculture',
       'vehicleClass_others', 'seatCapacity_1_to_3', 'seatCapacity_4_to_6',
       'seatCapacity_above_6', 'brand_new_vehicles', 'pre_owned_vehicles',
       'category_non_transport', 'category_transport', 'mmm', 'quarter'],
      dtype='object')

In [12]:
merged_df.describe()

,id_x,documents_registered_cnt,documents_registered_rev,estamps_challans_cnt,estamps_challans_rev,fiscal_year,id_y,investment_in_cr,number_of_employees,id,fuel_type_petrol,fuel_type_diesel,fuel_type_electric,fuel_type_others,vehicleClass_MotorCycle,vehicleClass_MotorCar,vehicleClass_AutoRickshaw,vehicleClass_Agriculture,vehicleClass_others,seatCapacity_1_to_3,seatCapacity_4_to_6,seatCapacity_above_6,brand_new_vehicles,pre_owned_vehicles,category_non_transport,category_transport
count,2651.000000,2651.000000,2.651000e+03,2651.000000,2.651000e+03,2651.000000,2651.000000,2651.000000,2651.000000,2651.000000,2651.000000,2651.000000,2651.000000,2651.000000,2651.000000,2651.000000,2651.000000,2651.000000,2651.000000,2651.000000,2651.000000,2651.000000,2651.000000,2651.000000,2651.000000,2651.000000
mean,1127.661260,5134.613731,5.307795e+08,5398.073934,5.358933e+08,2021.505470,4309.285553,15.317661,69.713316,1080.695964,4836.904942,781.574500,205.973218,100.437948,3948.456809,1338.044512,62.907959,119.219917,456.393059,4425.930592,1296.772539,197.257639,5375.679366,549.372690,5427.035458,498.016598
std,207.313721,5696.522302,9.389236e+08,6053.862795,9.501981e+08,0.500064,852.989505,90.733813,278.194448,203.284152,5955.959636,788.788318,391.723490,149.305224,4648.960210,2105.408534,98.288062,99.261607,546.735901,4975.262887,1922.165221,311.480857,6300.446284,870.487043,6691.588843,528.658747
min,769.000000,48.000000,7.201780e+05,46.000000,7.237980e+05,2021.000000,2809.000000,0.000000,0.000000,721.000000,229.000000,62.000000,0.000000,0.000000,207.000000,20.000000,0.000000,0.000000,31.000000,275.000000,26.000000,1.000000,316.000000,10.000000,252.000000,31.000000
25%,952.500000,1371.000000,3.724384e+07,1431.000000,3.855346e+07,2021.000000,3630.500000,0.250000,8.000000,921.500000,1450.500000,278.000000,16.000000,8.000000,1287.000000,203.000000,10.000000,48.000000,149.000000,1575.000000,221.000000,31.000000,1757.000000,88.000000,1632.500000,171.000000
50%,1126.000000,2877.000000,9.644171e+07,3024.000000,9.508621e+07,2022.000000,4336.000000,0.960000,16.000000,1086.000000,2185.000000,466.000000,39.000000,40.000000,1920.000000,353.000000,31.000000,96.000000,230.000000,2255.000000,393.000000,60.000000,2597.000000,144.000000,2434.000000,284.000000
75%,1293.000000,5666.000000,3.966681e+08,5997.000000,4.290958e+08,2022.000000,5036.500000,4.420000,46.000000,1248.000000,4410.000000,843.000000,154.500000,150.000000,3553.000000,988.000000,71.500000,155.000000,411.000000,4170.500000,1110.000000,147.000000,4941.000000,400.000000,4913.000000,546.000000
max,1504.000000,25965.000000,4.073496e+09,28135.000000,4.294273e+09,2022.000000,5753.000000,1862.730000,7000.000000,1440.000000,26093.000000,3245.000000,2606.000000,1848.000000,22499.000000,8036.000000,1684.000000,745.000000,2590.000000,23314.000000,7568.000000,1337.000000,26560.000000,4450.000000,27809.000000,3314.000000


In [13]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2651 entries, 0 to 2650
Data columns (total 35 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   id_x                       2651 non-null   int64         
 1   month                      2651 non-null   datetime64[ns]
 2   documents_registered_cnt   2651 non-null   int64         
 3   documents_registered_rev   2651 non-null   float64       
 4   estamps_challans_cnt       2651 non-null   int64         
 5   estamps_challans_rev       2651 non-null   float64       
 6   district                   2651 non-null   object        
 7   mmm_x                      2651 non-null   object        
 8   quarter_x                  2651 non-null   object        
 9   fiscal_year                2651 non-null   int64         
 10  id_y                       2651 non-null   int64         
 11  sector                     2651 non-null   object        
 12  invest

In [23]:
# Creating new columns for total revenue, total vehicle sales
merged_df['total_revenue'] = merged_df['documents_registered_rev'] + merged_df['estamps_challans_rev']
merged_df['total_vehicle_sales'] = merged_df[['fuel_type_petrol', 'fuel_type_diesel', 'fuel_type_electric', 'fuel_type_others']].sum(axis=1)

# Creating a new DataFrame with the selected columns
selected_columns = ['month', 'district', 'fiscal_year', 'quarter_x', 'mmm_x', 
                    'total_revenue', 'investment_in_cr', 'number_of_employees', 'total_vehicle_sales']

df = merged_df[selected_columns].copy()

# Renaming the columns for better readability
df.rename(columns={'quarter_x': 'quarter', 'mmm_x': 'mmm'}, inplace=True)

# Display the new DataFrame
df.head()

,month,district,fiscal_year,quarter,mmm,total_revenue,investment_in_cr,number_of_employees,total_vehicle_sales
0,2021-04-01,Yadadri Bhuvanagiri,2021,Q1,Apr,187927786.0,2.50,14,2164
1,2021-04-01,Yadadri Bhuvanagiri,2021,Q1,Apr,187927786.0,0.65,40,2164
2,2021-04-01,Yadadri Bhuvanagiri,2021,Q1,Apr,187927786.0,6.25,29,2164
3,2021-04-01,Yadadri Bhuvanagiri,2021,Q1,Apr,187927786.0,0.96,6,2164
4,2021-04-01,Yadadri Bhuvanagiri,2021,Q1,Apr,187927786.0,1.12,40,2164


In [25]:
# Columns to be normalized
columns_to_normalize = ['total_revenue', 'investment_in_cr', 'number_of_employees', 'total_vehicle_sales']

# Applying Min-Max Scaling
scaler = MinMaxScaler()
df[columns_to_normalize] = scaler.fit_transform(df[columns_to_normalize])

# Display the normalized DataFrame
df.head()

,month,district,fiscal_year,quarter,mmm,total_revenue,investment_in_cr,number_of_employees,total_vehicle_sales
0,2021-04-01,Yadadri Bhuvanagiri,2021,Q1,Apr,0.02229,0.001342,0.002000,0.060226
1,2021-04-01,Yadadri Bhuvanagiri,2021,Q1,Apr,0.02229,0.000349,0.005714,0.060226
2,2021-04-01,Yadadri Bhuvanagiri,2021,Q1,Apr,0.02229,0.003355,0.004143,0.060226
3,2021-04-01,Yadadri Bhuvanagiri,2021,Q1,Apr,0.02229,0.000515,0.000857,0.060226
4,2021-04-01,Yadadri Bhuvanagiri,2021,Q1,Apr,0.02229,0.000601,0.005714,0.060226
